**Load data as obtained from the Microplate reader and build a dataframe with the raw values**

In [ ]:
import pandas as pd

# Treatment labels (16 treatments × 4 replicates)
treatments = [
    "NC", "Dip8", "Dip16", "Dip32",
    "T-720", "T-360", "T-180",
    "Dip8/T-720", "Dip8/T-360", "Dip8/T-180",
    "Dip16/T-720", "Dip16/T-360", "Dip16/T-180",
    "Dip32/T-720", "Dip32/T-360", "Dip32/T-180"
]

# Load the 96 wells plate data
df_plate = pd.read_csv("dip_tuc_1.csv", header=None,sep=';')

# Extract only the data range: row 46–53 (index 45–52), columns 2–13 (index 1–12)
df_values = df_plate.iloc[45:53, 1:13]

# Clean and convert values to float, replacing commas
df_cleaned = df_values.map(
    lambda x: float(str(x).replace(",", ".")) if pd.notna(x) and "EP" not in str(x) else None
)

# Initialize result container
data = []

# Current treatment index
treatment_index = 0

# Loop over columns
for col in range(df_cleaned.shape[1]):
    col_values = df_cleaned.iloc[:, col]

    # Extract all valid (non-na) values from the column
    valid_vals = col_values.dropna().tolist()

    # Process in groups of 4
    for i in range(0, len(valid_vals), 4):
        reps = valid_vals[i:i+4]
        if len(reps) == 4 and treatment_index < len(treatments):
            data.append([treatments[treatment_index]] + reps)
            treatment_index += 1

# Build DataFrame with raw data
df_raw = pd.DataFrame(data, columns=["treatment", "rep1", "rep2", "rep3", "rep4"])

#save raw data if needed
#df_raw.to_csv("df_raw.csv", index=False)


**Build a dataframe adjusted by NC**

In [ ]:
import numpy as np

# Rename for clarity and to keep both files
df_plot = df_raw.copy()

# Calculate mean of the negative control (NC)
nc_values = df_plot[df_plot["treatment"] == "NC"].iloc[0, 1:5].values.astype(float)
nc_mean = np.mean(nc_values)

# Normalize OD values to viability
for rep in ["rep1", "rep2", "rep3", "rep4"]:
    df_plot[rep] = df_plot[rep].astype(float) / nc_mean * 100

# Add 'mean' and 'SD' columns
df_plot["mean"] = df_plot[["rep1", "rep2", "rep3", "rep4"]].mean(axis=1)
df_plot["SD"] = df_plot[["rep1", "rep2", "rep3", "rep4"]].std(axis=1)

# Preview
print(df_plot)

# Save the normalized data to plot
df_plot.to_csv("df_plot.csv", index=False)


**Built DataFrame for SynergyFinder**

In [ ]:
import re

# Prepare the long format list
synergy_rows = []

for _, row in df_plot.iterrows():
    treatment = row["treatment"]
    
    # Skip NC
    if treatment == "NC":
        continue

    # Identify type of treatment
    if "/" in treatment:  # Combination
        match = re.match(r"Dip(\d+)/T-(\d+)", treatment)
        if not match:
            continue
        conc1 = int(match.group(1))
        conc2 = int(match.group(2))
    elif "Dip" in treatment:  # Dipyridamole only
        match = re.match(r"Dip(\d+)", treatment)
        conc1 = int(match.group(1))
        conc2 = 0
    elif "T-" in treatment:  # Tucidinostat only
        match = re.match(r"T-(\d+)", treatment)
        conc1 = 0
        conc2 = int(match.group(1))
    else:
        continue  # skip anything else (like blanks)

    block_id = treatment
    for i in range(1, 5):  # rep1 to rep4
        response = row[f"rep{i}"]
        synergy_rows.append({
            "block_id": block_id,
            "drug1": "Dipyridamole",
            "drug2": "Tucidinostat",
            "conc1": conc1,
            "conc2": conc2,
            "response": response,
            "conc_unit1": "mM",
            "conc_unit2": "uM",
        })

# Create the synergy-ready DataFrame
df_synergy = pd.DataFrame(synergy_rows)

# Show the formatted data
print(df_synergy.head())

# Save to CSV for SynergyFinder Plus
df_synergy.to_csv("synergyfinder_input.csv", index=False)